In [1]:
import gc

import numpy as np
import pandas as pd
from scipy import sparse
import json


from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
import xgboost as xgb
from xgboost import XGBClassifier
from skopt import BayesSearchCV

from sklearn.model_selection import StratifiedKFold


def read_file(file_name, max_iters=float('Inf')):
    data = []
    n_items = 0

    with open(file_name, 'r') as f:
        for row in f:
            n_items += 1
            data.append(row)
            
            if n_items > max_iters:
                break
    return data

# Сохранение данных

### Датасет

In [ ]:
X_train = sparse.load_npz('..\\sparse\\sp_train_dataset.npz')

# Правильные ответы на train
y_train = read_file('..\\sparse\\dataset_s_labels.csv')
y_train = np.array([int(a) for a in y_train])
y_train[y_train==-1] = 0

X_train, y_train.shape

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtrain.save_binary('..\\sparse\\dtrain_dataset.bin')

In [ ]:
del X_train, y_train, dtrain
gc.collect()

### Test

In [ ]:
X_test = sparse.load_npz('..\\sparse\\sp_test_dataset.npz')
dtest = xgb.DMatrix(X_test)

dtest.save_binary('..\\sparse\\dtest_dataset.bin')

In [ ]:
del X_test, dtest
gc.collect()

# Загрузка данных

In [3]:
X_train = sparse.load_npz('..\\sparse\\sp_x_train_shuffled.npz')
X_valid = sparse.load_npz('..\\sparse\\sp_x_valid_shuffled.npz')

y_train = sparse.load_npz('..\\sparse\\sp_y_train_shuffled.npz')
y_valid = sparse.load_npz('..\\sparse\\sp_y_valid_shuffled.npz')

y_train = y_train.toarray()[0]
y_valid = y_valid.toarray()[0]

y_train[y_train==-1] = 0
y_valid[y_valid==-1] = 0

X_train.shape, y_train.shape

((385194, 793760), (385194,))

## Важность признаков

In [2]:
dtrain = xgb.DMatrix('..\\sparse\\dtrain_dataset.bin')

In [20]:
%%time

params = {
    'eta': 0.03,
    'max_depth': 10,
    'subsample': 0.5,
    'nthread': 4,
    'seed': 2000,
}

model = xgb.train(params, dtrain, 100, verbose_eval=1)

Wall time: 5min 51s


In [22]:
model.get_score()

{}

In [18]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12,18))
xgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)

ValueError: Booster.get_score() results in empty

# sklearn API

In [3]:
params = {
    'learning_rate': 0.1,
    'min_child_weight': 10,
    'max_depth': 10,
    'gamma': 0.3,
    'subsample': 0.9,
    'colsample_bytree': 0.7,
    'reg_lambda': 0.9,
    'reg_alpha': 0.5,
    
    'scale_pos_weight': 19,
#     'scale_pos_weight': 5,
    
    'objective': 'binary:logistic',
    'nthread': 10,
    
    
    'eval_metric': 'auc',
    'seed': 2000,
#     'tree_method': 'approx',
    'booster': 'gblinear',
        }

In [4]:
model = XGBClassifier(**params)

In [5]:
model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10)

[0]	validation_0-auc:0.566378
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.566378
[2]	validation_0-auc:0.566378
[3]	validation_0-auc:0.566378
[4]	validation_0-auc:0.566378
[5]	validation_0-auc:0.566378
[6]	validation_0-auc:0.566378
[7]	validation_0-auc:0.566378
[8]	validation_0-auc:0.566378
[9]	validation_0-auc:0.566378
[10]	validation_0-auc:0.566378
Stopping. Best iteration:
[0]	validation_0-auc:0.566378



XGBClassifier(base_score=0.5, booster='gblinear', colsample_bylevel=1,
       colsample_bytree=0.7, eval_metric='auc', gamma=0.3,
       learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=10, missing=None, n_estimators=100, n_jobs=1,
       nthread=10, objective='binary:logistic', random_state=0,
       reg_alpha=0.5, reg_lambda=0.9, scale_pos_weight=19, seed=2000,
       silent=True, subsample=0.9)

In [ ]:
y_pred = model.predict_proba(X_valid)
roc_auc_score(y_valid, y_pred)

## XGB

In [4]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)

del X_train, X_valid
gc.collect()

116

In [5]:
# watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
watchlist = [(dvalid, 'valid')]

In [22]:
# n_estimators=1000

params = {
    'eta': 0.1,
#     'min_child_weight': 1,
    'max_depth': 10,
#     'gamma': 0.5,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'lambda': 1.5,
    'alpha': 0.01,
    
#     'scale_pos_weight': 19,
    'scale_pos_weight': 50,
    
    'objective': 'binary:logistic',
    'nthread': 10,
    
    
    'eval_metric': 'auc',
    'seed': 2000,
#     'tree_method': 'approx',
    'booster': 'gblinear',
        }

In [23]:
# model = xgb.train(params, dtrain, 300, watchlist, maximize=True, early_stopping_rounds=10, verbose_eval=1)
model = xgb.train(params, dtrain, 300, watchlist, early_stopping_rounds=10, verbose_eval=1)

[0]	valid-auc:0.551959
Will train until valid-auc hasn't improved in 10 rounds.
[1]	valid-auc:0.551932
[2]	valid-auc:0.551892
[3]	valid-auc:0.551876
[4]	valid-auc:0.551872
[5]	valid-auc:0.551913
[6]	valid-auc:0.551972
[7]	valid-auc:0.552067
[8]	valid-auc:0.552166
[9]	valid-auc:0.552332
[10]	valid-auc:0.552556
[11]	valid-auc:0.55281
[12]	valid-auc:0.553156
[13]	valid-auc:0.553555
[14]	valid-auc:0.553969
[15]	valid-auc:0.554358
[16]	valid-auc:0.554736
[17]	valid-auc:0.555213
[18]	valid-auc:0.555576
[19]	valid-auc:0.555925
[20]	valid-auc:0.55636
[21]	valid-auc:0.556732
[22]	valid-auc:0.557105
[23]	valid-auc:0.557406
[24]	valid-auc:0.55768
[25]	valid-auc:0.557986
[26]	valid-auc:0.55819
[27]	valid-auc:0.558332
[28]	valid-auc:0.558637
[29]	valid-auc:0.558909
[30]	valid-auc:0.559106
[31]	valid-auc:0.55931
[32]	valid-auc:0.559518
[33]	valid-auc:0.559663
[34]	valid-auc:0.559813
[35]	valid-auc:0.560003
[36]	valid-auc:0.560109
[37]	valid-auc:0.560331
[38]	valid-auc:0.560478
[39]	valid-auc:0.56070

In [24]:
y_pred = model.predict(dvalid)
roc_auc_score(y_valid, y_pred)

0.5688300524138674

In [ ]:
output = pd.DataFrame({ 'activity_id' : test['activity_id'], 'outcome': ypred })
output.head()
output.to_csv('without_leak.csv', index = False)

## Baseline

In [10]:
dtrain = xgb.DMatrix('..\\sparse\\dtrain_dataset.bin')
dtest = xgb.DMatrix('..\\sparse\\dtest_dataset.bin')

In [2]:
params = {
    'eta': 0.01,
#     'min_child_weight': 1,
    'max_depth': 6,
#     'gamma': 0.5,
#     'subsample': 0.7,
#     'colsample_bytree': 0.7,
#     'lambda': 1.5,
#     'alpha': 0.01,
    
    'scale_pos_weight': 19,
#     'scale_pos_weight': 50,
    
    'objective': 'binary:logistic',
    'nthread': 10,
    
    'eval_metric': 'auc',
    'seed': 2000,
#     'tree_method': 'approx',
    'booster': 'gblinear',
}

In [4]:
watchlist = [(dtrain, 'train')]

In [8]:
%%time
model = xgb.train(params, dtrain, 3000, watchlist, verbose_eval=1)

[0]	train-auc:0.534715
[1]	train-auc:0.535499
[2]	train-auc:0.536811
[3]	train-auc:0.537938
[4]	train-auc:0.538894
[5]	train-auc:0.539667
[6]	train-auc:0.540296
[7]	train-auc:0.540793
[8]	train-auc:0.541216
[9]	train-auc:0.541553
[10]	train-auc:0.541843
[11]	train-auc:0.542086
[12]	train-auc:0.542296
[13]	train-auc:0.54248
[14]	train-auc:0.542641
[15]	train-auc:0.542787
[16]	train-auc:0.542917
[17]	train-auc:0.543035
[18]	train-auc:0.543142
[19]	train-auc:0.54324
[20]	train-auc:0.543326
[21]	train-auc:0.543404
[22]	train-auc:0.543478
[23]	train-auc:0.543544
[24]	train-auc:0.543596
[25]	train-auc:0.543655
[26]	train-auc:0.543706
[27]	train-auc:0.543748
[28]	train-auc:0.543769
[29]	train-auc:0.543801
[30]	train-auc:0.543829
[31]	train-auc:0.543845
[32]	train-auc:0.543901
[33]	train-auc:0.543913
[34]	train-auc:0.543938
[35]	train-auc:0.543842
[36]	train-auc:0.543844
[37]	train-auc:0.543883
[38]	train-auc:0.543847
[39]	train-auc:0.543733
[40]	train-auc:0.543681
[41]	train-auc:0.543554
[42]

[334]	train-auc:0.524235
[335]	train-auc:0.52413
[336]	train-auc:0.524022
[337]	train-auc:0.523988
[338]	train-auc:0.523961
[339]	train-auc:0.523996
[340]	train-auc:0.524003
[341]	train-auc:0.524033
[342]	train-auc:0.523961
[343]	train-auc:0.523971
[344]	train-auc:0.524015
[345]	train-auc:0.524037
[346]	train-auc:0.52403
[347]	train-auc:0.524086
[348]	train-auc:0.524065
[349]	train-auc:0.524009
[350]	train-auc:0.523991
[351]	train-auc:0.524021
[352]	train-auc:0.524048
[353]	train-auc:0.524041
[354]	train-auc:0.524025
[355]	train-auc:0.52398
[356]	train-auc:0.523984
[357]	train-auc:0.524004
[358]	train-auc:0.524038
[359]	train-auc:0.524028
[360]	train-auc:0.52403
[361]	train-auc:0.524027
[362]	train-auc:0.524037
[363]	train-auc:0.524073
[364]	train-auc:0.524118
[365]	train-auc:0.524174
[366]	train-auc:0.52421
[367]	train-auc:0.524147
[368]	train-auc:0.524147
[369]	train-auc:0.524121
[370]	train-auc:0.52406
[371]	train-auc:0.524087
[372]	train-auc:0.524114
[373]	train-auc:0.524105
[374]	

[664]	train-auc:0.521323
[665]	train-auc:0.521327
[666]	train-auc:0.521313
[667]	train-auc:0.521322
[668]	train-auc:0.521306
[669]	train-auc:0.521291
[670]	train-auc:0.521281
[671]	train-auc:0.52127
[672]	train-auc:0.521279
[673]	train-auc:0.521284
[674]	train-auc:0.521288
[675]	train-auc:0.521293
[676]	train-auc:0.521297
[677]	train-auc:0.521266
[678]	train-auc:0.521256
[679]	train-auc:0.521264
[680]	train-auc:0.521267
[681]	train-auc:0.521276
[682]	train-auc:0.521284
[683]	train-auc:0.521248
[684]	train-auc:0.521235
[685]	train-auc:0.521244
[686]	train-auc:0.521235
[687]	train-auc:0.521239
[688]	train-auc:0.521246
[689]	train-auc:0.521253
[690]	train-auc:0.521238
[691]	train-auc:0.52124
[692]	train-auc:0.521246
[693]	train-auc:0.521252
[694]	train-auc:0.52126
[695]	train-auc:0.521269
[696]	train-auc:0.521256
[697]	train-auc:0.521257
[698]	train-auc:0.521263
[699]	train-auc:0.521267
[700]	train-auc:0.521272
[701]	train-auc:0.521276
[702]	train-auc:0.521285
[703]	train-auc:0.521289
[70

[994]	train-auc:0.521119
[995]	train-auc:0.521123
[996]	train-auc:0.52111
[997]	train-auc:0.521116
[998]	train-auc:0.521127
[999]	train-auc:0.521135
[1000]	train-auc:0.521141
[1001]	train-auc:0.521141
[1002]	train-auc:0.521111
[1003]	train-auc:0.521118
[1004]	train-auc:0.521129
[1005]	train-auc:0.52112
[1006]	train-auc:0.521124
[1007]	train-auc:0.521131
[1008]	train-auc:0.521141
[1009]	train-auc:0.521147
[1010]	train-auc:0.52114
[1011]	train-auc:0.521149
[1012]	train-auc:0.521156
[1013]	train-auc:0.521126
[1014]	train-auc:0.521135
[1015]	train-auc:0.521141
[1016]	train-auc:0.521126
[1017]	train-auc:0.521128
[1018]	train-auc:0.521113
[1019]	train-auc:0.521102
[1020]	train-auc:0.521113
[1021]	train-auc:0.521085
[1022]	train-auc:0.521097
[1023]	train-auc:0.521101
[1024]	train-auc:0.521111
[1025]	train-auc:0.521101
[1026]	train-auc:0.521108
[1027]	train-auc:0.521096
[1028]	train-auc:0.521102
[1029]	train-auc:0.521089
[1030]	train-auc:0.521076
[1031]	train-auc:0.521065
[1032]	train-auc:0.52

[1311]	train-auc:0.520388
[1312]	train-auc:0.520375
[1313]	train-auc:0.520378
[1314]	train-auc:0.520385
[1315]	train-auc:0.520391
[1316]	train-auc:0.520399
[1317]	train-auc:0.520403
[1318]	train-auc:0.520412
[1319]	train-auc:0.520421
[1320]	train-auc:0.520424
[1321]	train-auc:0.52041
[1322]	train-auc:0.520417
[1323]	train-auc:0.520421
[1324]	train-auc:0.52041
[1325]	train-auc:0.520415
[1326]	train-auc:0.520425
[1327]	train-auc:0.520436
[1328]	train-auc:0.520422
[1329]	train-auc:0.520388
[1330]	train-auc:0.52039
[1331]	train-auc:0.520397
[1332]	train-auc:0.520383
[1333]	train-auc:0.520386
[1334]	train-auc:0.520397
[1335]	train-auc:0.520384
[1336]	train-auc:0.520329
[1337]	train-auc:0.520316
[1338]	train-auc:0.520323
[1339]	train-auc:0.520327
[1340]	train-auc:0.520295
[1341]	train-auc:0.520301
[1342]	train-auc:0.520306
[1343]	train-auc:0.520291
[1344]	train-auc:0.520297
[1345]	train-auc:0.520306
[1346]	train-auc:0.520311
[1347]	train-auc:0.520322
[1348]	train-auc:0.520312
[1349]	train-au

[1628]	train-auc:0.519878
[1629]	train-auc:0.519885
[1630]	train-auc:0.519891
[1631]	train-auc:0.519857
[1632]	train-auc:0.519841
[1633]	train-auc:0.519845
[1634]	train-auc:0.519836
[1635]	train-auc:0.519823
[1636]	train-auc:0.519832
[1637]	train-auc:0.519838
[1638]	train-auc:0.519841
[1639]	train-auc:0.519827
[1640]	train-auc:0.519832
[1641]	train-auc:0.519836
[1642]	train-auc:0.519849
[1643]	train-auc:0.519835
[1644]	train-auc:0.519821
[1645]	train-auc:0.519824
[1646]	train-auc:0.519834
[1647]	train-auc:0.519822
[1648]	train-auc:0.519808
[1649]	train-auc:0.519814
[1650]	train-auc:0.519819
[1651]	train-auc:0.519826
[1652]	train-auc:0.519813
[1653]	train-auc:0.5198
[1654]	train-auc:0.519786
[1655]	train-auc:0.519773
[1656]	train-auc:0.51978
[1657]	train-auc:0.519788
[1658]	train-auc:0.519795
[1659]	train-auc:0.519798
[1660]	train-auc:0.519804
[1661]	train-auc:0.51981
[1662]	train-auc:0.519815
[1663]	train-auc:0.519783
[1664]	train-auc:0.519776
[1665]	train-auc:0.519784
[1666]	train-auc

[1945]	train-auc:0.518979
[1946]	train-auc:0.518963
[1947]	train-auc:0.518971
[1948]	train-auc:0.518975
[1949]	train-auc:0.518963
[1950]	train-auc:0.518966
[1951]	train-auc:0.51895
[1952]	train-auc:0.518951
[1953]	train-auc:0.518954
[1954]	train-auc:0.518955
[1955]	train-auc:0.518961
[1956]	train-auc:0.518946
[1957]	train-auc:0.518949
[1958]	train-auc:0.518953
[1959]	train-auc:0.518955
[1960]	train-auc:0.518956
[1961]	train-auc:0.518961
[1962]	train-auc:0.518963
[1963]	train-auc:0.518966
[1964]	train-auc:0.51897
[1965]	train-auc:0.518974
[1966]	train-auc:0.518981
[1967]	train-auc:0.518987
[1968]	train-auc:0.51897
[1969]	train-auc:0.518972
[1970]	train-auc:0.518974
[1971]	train-auc:0.518979
[1972]	train-auc:0.518983
[1973]	train-auc:0.518987
[1974]	train-auc:0.51899
[1975]	train-auc:0.518992
[1976]	train-auc:0.518996
[1977]	train-auc:0.518981
[1978]	train-auc:0.518983
[1979]	train-auc:0.518966
[1980]	train-auc:0.518972
[1981]	train-auc:0.518976
[1982]	train-auc:0.51898
[1983]	train-auc:

[2262]	train-auc:0.518718
[2263]	train-auc:0.51872
[2264]	train-auc:0.518724
[2265]	train-auc:0.518725
[2266]	train-auc:0.518728
[2267]	train-auc:0.518728
[2268]	train-auc:0.518729
[2269]	train-auc:0.518732
[2270]	train-auc:0.518732
[2271]	train-auc:0.518733
[2272]	train-auc:0.518733
[2273]	train-auc:0.518734
[2274]	train-auc:0.518718
[2275]	train-auc:0.518721
[2276]	train-auc:0.518722
[2277]	train-auc:0.518722
[2278]	train-auc:0.518722
[2279]	train-auc:0.518723
[2280]	train-auc:0.518723
[2281]	train-auc:0.518727
[2282]	train-auc:0.518728
[2283]	train-auc:0.518732
[2284]	train-auc:0.518736
[2285]	train-auc:0.518738
[2286]	train-auc:0.518742
[2287]	train-auc:0.518744
[2288]	train-auc:0.518727
[2289]	train-auc:0.518732
[2290]	train-auc:0.518735
[2291]	train-auc:0.518735
[2292]	train-auc:0.518716
[2293]	train-auc:0.518718
[2294]	train-auc:0.518722
[2295]	train-auc:0.518722
[2296]	train-auc:0.518723
[2297]	train-auc:0.518723
[2298]	train-auc:0.518723
[2299]	train-auc:0.518724
[2300]	train-

[2579]	train-auc:0.518658
[2580]	train-auc:0.518658
[2581]	train-auc:0.518659
[2582]	train-auc:0.518663
[2583]	train-auc:0.518664
[2584]	train-auc:0.518665
[2585]	train-auc:0.518666
[2586]	train-auc:0.518668
[2587]	train-auc:0.518669
[2588]	train-auc:0.518671
[2589]	train-auc:0.518674
[2590]	train-auc:0.518675
[2591]	train-auc:0.518676
[2592]	train-auc:0.518678
[2593]	train-auc:0.518658
[2594]	train-auc:0.518659
[2595]	train-auc:0.518663
[2596]	train-auc:0.518664
[2597]	train-auc:0.518666
[2598]	train-auc:0.518666
[2599]	train-auc:0.518647
[2600]	train-auc:0.518649
[2601]	train-auc:0.518651
[2602]	train-auc:0.518651
[2603]	train-auc:0.518652
[2604]	train-auc:0.518652
[2605]	train-auc:0.518652
[2606]	train-auc:0.518632
[2607]	train-auc:0.518633
[2608]	train-auc:0.518634
[2609]	train-auc:0.518634
[2610]	train-auc:0.518635
[2611]	train-auc:0.518635
[2612]	train-auc:0.518635
[2613]	train-auc:0.518635
[2614]	train-auc:0.518635
[2615]	train-auc:0.518635
[2616]	train-auc:0.518635
[2617]	train

[2896]	train-auc:0.518544
[2897]	train-auc:0.518545
[2898]	train-auc:0.518545
[2899]	train-auc:0.518545
[2900]	train-auc:0.518545
[2901]	train-auc:0.518545
[2902]	train-auc:0.518545
[2903]	train-auc:0.518545
[2904]	train-auc:0.518547
[2905]	train-auc:0.518548
[2906]	train-auc:0.518548
[2907]	train-auc:0.518549
[2908]	train-auc:0.518549
[2909]	train-auc:0.51855
[2910]	train-auc:0.51855
[2911]	train-auc:0.518551
[2912]	train-auc:0.518551
[2913]	train-auc:0.518552
[2914]	train-auc:0.518552
[2915]	train-auc:0.518552
[2916]	train-auc:0.518553
[2917]	train-auc:0.518554
[2918]	train-auc:0.518554
[2919]	train-auc:0.518537
[2920]	train-auc:0.518537
[2921]	train-auc:0.518538
[2922]	train-auc:0.518539
[2923]	train-auc:0.518539
[2924]	train-auc:0.51854
[2925]	train-auc:0.518542
[2926]	train-auc:0.518544
[2927]	train-auc:0.518545
[2928]	train-auc:0.518545
[2929]	train-auc:0.518546
[2930]	train-auc:0.518546
[2931]	train-auc:0.518548
[2932]	train-auc:0.518549
[2933]	train-auc:0.518549
[2934]	train-au

In [11]:
y_pred = model.predict(dtest)

### Save submission

In [12]:
cuids_out = pd.read_csv('..\\input\\mlboot_test.tsv', index_col=[0])

In [13]:
submission = pd.read_csv('..\\sparse\\dataset_s_test_cuid.csv', index_col=[0], names=['cuid'], header=None)
submission['pred'] = y_pred
submission = submission.reindex(cuids_out.index)

submission.to_csv('..\\output\\sp_baseline_xgb.csv', header=None, index=False)

# Optimization

In [2]:
X_train = sparse.load_npz('..\\sparse\\sp_x_train_shuffled.npz')

y_train = sparse.load_npz('..\\sparse\\sp_y_train_shuffled.npz')
y_train = y_train.toarray()[0]
y_train[y_train==-1] = 0

In [3]:
ITERATIONS = 100 # 1000

# Classifier
bayes_cv_tuner = BayesSearchCV(
#     estimator = xgb.XGBClassifier(n_jobs=1, objective='binary:logistic', eval_metric='auc', silent=1, tree_method='approx'),
    estimator = xgb.XGBClassifier(n_jobs=1, objective='binary:logistic', eval_metric='auc', silent=1),
    search_spaces = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'min_child_weight': (0, 5),
        'max_depth': (0, 50),
        'max_delta_step': (0, 20),
        'subsample': (0.01, 1.0, 'uniform'),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'colsample_bylevel': (0.01, 1.0, 'uniform'),
        'reg_lambda': (1e-9, 1000, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'gamma': (1e-9, 0.5, 'log-uniform'),
        'n_estimators': (50, 100),
        'scale_pos_weight': (1, 100, 'log-uniform')
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42),
    n_jobs = -1,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = False, #True,
    random_state = 42
)

def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(clf_name+"_cv_results.csv")

In [ ]:
# Fit the model
result = bayes_cv_tuner.fit(X_train, y_train, callback=status_print)